<a href="https://colab.research.google.com/github/Subru97/Fitness-activity-recognition/blob/master/Fitness_activity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wEidOQlvHSa0woLjnAO7ZrE1J67LLHA8ixFU-8MEl0xvQAXukVcP84
Mounted at /content/drive/


In [7]:
!ls '/content/drive/My Drive/video_data'

JumpingJack  JumpRope  Lunges  PullUps	PushUps


In [24]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
 
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report

In [36]:
data_dir = "/content/drive/My Drive/video_data"
img_height , img_width = 64, 64
seq_len = 70
 
classes = ["JumpingJack", "JumpRope", "Lunges", "PullUps", "PushUps"]

In [11]:
def frames_extraction(video_path):
    frames_list = []
    
    vidObj = cv2.VideoCapture(video_path)
    # Used as counter variable 
    count = 1
 
    while count <= seq_len: 
         
        success, image = vidObj.read() 
        if success:
            image = cv2.resize(image, (img_height, img_width))
            frames_list.append(image)
            count += 1
        else:
            print("Defected frame")
            break
 
            
    return frames_list

In [12]:
def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = os.listdir(input_dir)
     
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list:
            frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
            if len(frames) == seq_len:
                X.append(frames)
             
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
     
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [13]:
model = Sequential()
model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), return_sequences = False, data_format = "channels_last", input_shape = (seq_len, img_height, img_width, 3)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(5, activation = "softmax"))

In [14]:
opt = keras.optimizers.SGD(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [15]:
earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]

In [38]:
X, Y = create_data(data_dir)
 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle=True, random_state=0)

PushUps
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Lunges
Defected frame
Defected frame
Defected frame
Defected frame
JumpingJack
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
Defected frame
PullUps
Defected frame
Defected frame
JumpRope


In [20]:
X_train.shape

(468, 70, 64, 64, 3)

In [21]:
history = model.fit(x = X_train, y = y_train, epochs=20, batch_size = 8 , shuffle=True, validation_split=0.2, callbacks=callbacks)

Epoch 1/20
47/47 [==============================] - 47s 999ms/step - loss: 1.6748 - accuracy: 0.2674 - val_loss: 1.4795 - val_accuracy: 0.3830
Epoch 2/20
47/47 [==============================] - 47s 1s/step - loss: 1.4838 - accuracy: 0.3610 - val_loss: 1.5463 - val_accuracy: 0.2234
Epoch 3/20
47/47 [==============================] - 48s 1s/step - loss: 1.3158 - accuracy: 0.4706 - val_loss: 1.0442 - val_accuracy: 0.6702
Epoch 4/20
47/47 [==============================] - 48s 1s/step - loss: 1.0356 - accuracy: 0.5882 - val_loss: 1.4270 - val_accuracy: 0.4787
Epoch 5/20
47/47 [==============================] - 48s 1s/step - loss: 0.9091 - accuracy: 0.6551 - val_loss: 1.0872 - val_accuracy: 0.5426
Epoch 6/20
47/47 [==============================] - 48s 1s/step - loss: 0.5603 - accuracy: 0.8128 - val_loss: 0.5160 - val_accuracy: 0.8936
Epoch 7/20
47/47 [==============================] - 48s 1s/step - loss: 0.3419 - accuracy: 0.9091 - val_loss: 0.5059 - val_accuracy: 0.7979
Epoch 8/20
47/47 

In [39]:
y_test.shape

(118, 5)

In [40]:
y_pred = model.predict(X_test)

y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)
 
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.86      0.90        21
           1       0.97      0.94      0.95        32
           2       0.88      1.00      0.94        23
           3       0.88      0.92      0.90        25
           4       0.94      0.88      0.91        17

    accuracy                           0.92       118
   macro avg       0.92      0.92      0.92       118
weighted avg       0.93      0.92      0.92       118



In [23]:
model.save('/content/drive/My Drive/video_data/model.h5')
model.save_weights('/content/drive/My Drive/video_data/weights.h5')

In [42]:
y_pred[50], y_test[50]

(4, 4)